# Scratch

---

### Imports & Setup

Installs

In [1]:
# Run this cell to install required packages in your Jupyter notebook environment
%pip install openai sqlalchemy sqlmodel psycopg2-binary pandas

Note: you may need to restart the kernel to use updated packages.


General imports

In [2]:
import os
from datetime import datetime, timedelta
from typing import List, Optional

Connect to DB

In [3]:
from sqlalchemy import create_engine
from sqlmodel import Session, SQLModel, select

# PostgreSQL connection URL
DATABASE_HOST = "localhost"  # Service name in docker-compose.yml
DATABASE_PORT = "5432"
DATABASE_NAME = "updatesdb"
DATABASE_USER = "postgres"
DATABASE_PASSWORD = os.getenv("DATABASE_PASSWORD")

ENGINE_URL = f"postgresql://{DATABASE_USER}:{DATABASE_PASSWORD}@{DATABASE_HOST}:{DATABASE_PORT}/{DATABASE_NAME}"

engine = create_engine(ENGINE_URL, echo=True, pool_pre_ping=True)

# Optional: If you need to create the tables in your database (uncomment if necessary)
# SQLModel.metadata.create_all(engine)

Models

In [4]:
from sqlmodel import SQLModel, Field, Relationship
from sqlalchemy import Column, BigInteger

class ChatType(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    type_name: str = Field(sa_column_kwargs={"unique": False})

class Chat(SQLModel, table=True):
    id: int = Field(primary_key=True)
    chat_id: Optional[int] = Field(default=None, sa_column=Column(BigInteger()))
    all_members_are_administrators: bool = Field()
    title: str = Field()
    type_id: int = Field(foreign_key="chattype.id")
    type: ChatType = Relationship()

class User(SQLModel, table=True):
    id: int = Field(primary_key=True, index=True)
    user_id: Optional[int] = Field(default=None)  # Telegram user ID
    first_name: str = Field()
    last_name: Optional[str] = Field(default=None)
    is_bot: bool = Field()
    language_code: str = Field()
    username: Optional[str] = Field()

class Message(SQLModel, table=True):
    id: int = Field(primary_key=True)
    message_id: int = Field(default=None)
    channel_chat_created: bool = Field()
    chat_id: int = Field(foreign_key="chat.id")
    chat: Chat = Relationship()
    date: int = Field()
    delete_chat_photo: bool = Field()
    from_user_id: int = Field(foreign_key="user.id")
    from_user: User = Relationship()
    group_chat_created: bool = Field()
    reply_to_message_id: Optional[int] = Field(default=None, foreign_key="message.id")
    reply_to_message: 'Message' = Relationship(sa_relationship_kwargs={"remote_side": "Message.id"})
    supergroup_chat_created: bool = Field()
    text: Optional[str] = Field()

class Update(SQLModel, table=True):
    update_id: int = Field(primary_key=True)
    message_id: Optional[int] = Field(default=None, foreign_key="message.id")
    message: Optional[Message] = Relationship()

---

### Display Tables (in pandas)

In [5]:
import pandas as pd

def display_table_data_pandas(model_class):
    with Session(engine) as session:
        statement = select(model_class)
        results = session.exec(statement).all()
        return pd.DataFrame([result.__dict__ for result in results])

In [6]:
# Display data using pandas for better formatting
data = display_table_data_pandas(Message)
data.head()

2024-05-13 10:30:23,296 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-05-13 10:30:23,297 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-05-13 10:30:23,298 INFO sqlalchemy.engine.Engine select current_schema()
2024-05-13 10:30:23,298 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-05-13 10:30:23,299 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-05-13 10:30:23,300 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-05-13 10:30:23,301 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-13 10:30:23,313 INFO sqlalchemy.engine.Engine SELECT message.id, message.message_id, message.channel_chat_created, message.chat_id, message.date, message.delete_chat_photo, message.from_user_id, message.group_chat_created, message.reply_to_message_id, message.supergroup_chat_created, message.text 
FROM message
2024-05-13 10:30:23,314 INFO sqlalchemy.engine.Engine [generated in 0.00059s] {}
2024-05-13 10:30:23,319 INFO sqlalchemy.engine.Engine ROLLBACK


,_sa_instance_state,message_id,channel_chat_created,date,from_user_id,reply_to_message_id,text,id,chat_id,delete_chat_photo,group_chat_created,supergroup_chat_created
0,<sqlalchemy.orm.state.InstanceState object at ...,22497,False,1715268507,2,NaN,are you saying this due to the whole wallet wh...,1,2,False,False,False
1,<sqlalchemy.orm.state.InstanceState object at ...,22501,False,1715270079,1,1.0,No. the same whitelisting issues exists with 7...,2,1,False,False,False
2,<sqlalchemy.orm.state.InstanceState object at ...,248,False,1715270331,3,NaN,"Ok Archiving is working fine for now, will sta...",3,3,False,False,False
3,<sqlalchemy.orm.state.InstanceState object at ...,22502,False,1715270488,4,NaN,"From the POV of wallets, I think the whitelist...",4,4,False,False,False
4,<sqlalchemy.orm.state.InstanceState object at ...,22502,False,1715270488,6,NaN,"From the POV of wallets, I think the whitelist...",5,6,False,False,False


---

### Reconstruct Chat History

This function takes a list of conversations and message and returns the index of the conversation the message belongs to, or none

In [12]:
from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()

def select_conversation_for_message(conversations: List[List[Message]], message: Message, gpt) -> int:
    # Prompt template
    prompt = """
This is a list of indexed conversations:

{conversations}

This is a message that needs to be placed in one of the conversations above:

{message}

Please reply with the index of the conversation that the message belongs to.
Think about the context of the conversation and how the message fits in.
Your response will be converted into an int for further processing, so please 
reply with only the number and absolutely nothing else.
"""

    conversation_text = ""

    for index, conversation in enumerate(conversations):
        # Generate the conversation text with an index
        conversation_text = conversation_text + f"\n{index}: \n" + "\n".join([f"{message.from_user.username}: {message.text}" for message in conversation])

    prompt = prompt.format(conversations=conversation_text, message=f"{message.from_user.username}: {message.text}")

    # Generate the completion
    completion = gpt.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=10,
        temperature=0.1,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    
    # Extract the completion choice
    choice = int(completion.choices[0].message.content.strip())

    # # Return the choice if it is valid
    if 0 <= choice < len(conversations):
        return choice
        
    return None

In [14]:
# Dummy conversations for testing
# Creating dummy users
user1 = User(id=1, username="Alice")
user2 = User(id=2, username="Bob")

# Creating dummy messages
message1 = Message(id=1, from_user=user1, text="Hello, how are you?", reply_to_message_id=None)
message2 = Message(id=2, from_user=user2, text="I'm good, thanks! And you?", reply_to_message_id=1)
message3 = Message(id=3, from_user=user1, text="Great to hear that, I'm fine too.", reply_to_message_id=2)

message4 = Message(id=4, from_user=user2, text="Have you seen the latest movie?", reply_to_message_id=None)
message5 = Message(id=5, from_user=user1, text="Not yet, is it good?", reply_to_message_id=4)

# Creating test message
# message = Message(id=6, from_user=user2, text="It was an amazing movie!", reply_to_message_id=None)
message = Message(id=6, from_user=user2, text="Ok, see you later!", reply_to_message_id=None)

# Organizing messages into conversations
conversations = [
    [message1, message2, message3],  # Conversation 1
    [message4, message5]             # Conversation 2
]

# Instantiate OpenAI chat client
gpt = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


# Test the function
select_conversation_for_message(conversations, message, gpt)

0


This function takes a list of conversations and a message and appends the message to the conversation it belongs to. 

In [17]:
def select_conversation_for_message_and_add(conversations: List[List[Message]], message: Message, gpt):
    # Check if the message is a direct reply
    if hasattr(message, 'reply_to_message_id') and message.reply_to_message_id:
        for conversation in conversations:
            for msg in conversation:
                if msg.id == message.reply_to_message_id:
                    conversation.append(message)
                    return
    
    # Use AI to find the right conversation for non-reply messages
    if len(conversations) > 0:
        conversation_index = select_conversation_for_message(conversations, message, gpt)
        if conversation_index is not None:
            conversations[conversation_index].append(message)
            return

    # If no conversation fits or list is empty, start a new conversation
    conversations.append([message])

In [18]:
def reconstruct_chat_as_conversations(engine, start_time: datetime, end_time: datetime, gpt) -> List[List[Message]]:
    conversations = []  # This will store lists of messages categorized into conversations
    with Session(engine) as session:
        # Fetch all messages within the given time range, ordering by the date
        statement = select(Message).where(
            Message.date >= start_time.timestamp(),  # Assuming 'date' is stored as a UNIX timestamp
            Message.date <= end_time.timestamp()
        ).order_by(Message.date)
        results = session.exec(statement).all()

        for message in results:
            select_conversation_for_message_and_add(conversations, message, gpt)
    
    return conversations

In [24]:
# Get dates
today = datetime.now()
previous_day = today - timedelta(days=5)
# Set start_time to the beginning of the previous day (00:00:00)
start_time = datetime(previous_day.year, previous_day.month, previous_day.day, 0, 0, 0)
# Set end_time to the end of the previous day (23:59:59)
end_time = datetime(today.year, today.month, today.day, 0, 0, 0)

# Get chat history text
conversations = reconstruct_chat_as_conversations(engine, start_time, end_time, gpt)

2024-05-13 10:46:31,834 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-13 10:46:31,834 INFO sqlalchemy.engine.Engine SELECT message.id, message.message_id, message.channel_chat_created, message.chat_id, message.date, message.delete_chat_photo, message.from_user_id, message.group_chat_created, message.reply_to_message_id, message.supergroup_chat_created, message.text 
FROM message 
WHERE message.date >= %(date_1)s AND message.date <= %(date_2)s ORDER BY message.date
2024-05-13 10:46:31,834 INFO sqlalchemy.engine.Engine [cached since 932.8s ago] {'date_1': 1715122800.0, 'date_2': 1715554800.0}
2024-05-13 10:46:31,836 INFO sqlalchemy.engine.Engine SELECT "user".id AS user_id_1, "user".user_id AS user_user_id, "user".first_name AS user_first_name, "user".last_name AS user_last_name, "user".is_bot AS user_is_bot, "user".language_code AS user_language_code, "user".username AS user_username 
FROM "user" 
WHERE "user".id = %(pk_1)s
2024-05-13 10:46:31,836 INFO sqlalchemy.engine.Engine 

In [26]:
conversations

[[Message(message_id=22351, channel_chat_created=False, date=1715146058, from_user_id=19, reply_to_message_id=None, text='The main value of code immutability is so that other contracts can point to specific addresses on chain and use them as libraries. And in ERC4337, accessing mutable libraries becomes a DoS attack vector', id=18, chat_id=19, delete_chat_photo=False, group_chat_created=False, supergroup_chat_created=False),
  Message(message_id=22497, channel_chat_created=False, date=1715268507, from_user_id=2, reply_to_message_id=None, text='are you saying this due to the whole wallet whitelisting debate?', id=1, chat_id=2, delete_chat_photo=False, group_chat_created=False, supergroup_chat_created=False),
  Message(message_id=22501, channel_chat_created=False, date=1715270079, from_user_id=1, reply_to_message_id=1, text='No. the same whitelisting issues exists with 7702 and 3074: You still fully trust your wallet to decide what code is safe to sign and what not.\n\nThe fact a single 

---

### Extract topics

Function to create chat text from list of messages

In [40]:
def create_transcripts_from_conversations(conversations: List[List[Message]], engine) -> str:
    transcripts = []
    for conversation in conversations:
        transcript = ""
        for message in conversation:
            # Check if the message is a reply
            if message.reply_to_message_id:
                # Fetch the message being replied to
                with Session(engine) as session:
                    statement = select(Message).where(Message.id == message.reply_to_message_id)
                    result = session.exec(statement).first()
                    transcript += f"{message.from_user.username} (replying to {result.from_user.username}): {message.text}\n"
            else:
                transcript += f"{message.from_user.username}: {message.text}\n"
        transcripts.append(transcript)
    return transcripts

In [41]:
transcripts = create_transcripts_from_conversations(conversations, engine)

2024-05-13 11:04:48,551 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-13 11:04:48,552 INFO sqlalchemy.engine.Engine SELECT message.id, message.message_id, message.channel_chat_created, message.chat_id, message.date, message.delete_chat_photo, message.from_user_id, message.group_chat_created, message.reply_to_message_id, message.supergroup_chat_created, message.text 
FROM message 
WHERE message.id = %(id_1)s
2024-05-13 11:04:48,552 INFO sqlalchemy.engine.Engine [cached since 119.6s ago] {'id_1': 1}
2024-05-13 11:04:48,554 INFO sqlalchemy.engine.Engine SELECT "user".id AS user_id_1, "user".user_id AS user_user_id, "user".first_name AS user_first_name, "user".last_name AS user_last_name, "user".is_bot AS user_is_bot, "user".language_code AS user_language_code, "user".username AS user_username 
FROM "user" 
WHERE "user".id = %(pk_1)s
2024-05-13 11:04:48,554 INFO sqlalchemy.engine.Engine [cached since 1806s ago] {'pk_1': 2}
2024-05-13 11:04:48,556 INFO sqlalchemy.engine.Engine ROLL

In [44]:
print(transcripts[0])

vb271828: The main value of code immutability is so that other contracts can point to specific addresses on chain and use them as libraries. And in ERC4337, accessing mutable libraries becomes a DoS attack vector
julrach: are you saying this due to the whole wallet whitelisting debate?
drortirosh (replying to julrach): No. the same whitelisting issues exists with 7702 and 3074: You still fully trust your wallet to decide what code is safe to sign and what not.

The fact a single transaction signed by account "X" can use and modify many other accounts which may (or may not) attempt to submit transactions of their own - this causes a problem for block-builder to determine the validity of those transaction. They have to simulate them completely, before deciding some should be dropped. Carefully crafted transactions can cause a block builder to spend a lot of time handling and dropping transactions, to a point it can't create an optimal block.
These problems are mitigated by 7702, since yo

In [94]:
from openai import OpenAI

def summarize_chat(chat_text, gpt):
    # Using the chat-specific completion API
    response = gpt.chat.completions.create(
        model="gpt-3.5-turbo",  # Use an appropriate model, like 'gpt-3.5-turbo'
        messages=[{"role": "system", "content": """
Provide a detailed summary of the chat transcript below.
The summary should capture the key points and essence of the conversation.
When summarizing, ensure you reference the participants by name and their specific positions.
Finish with a succinct conclusion that encapsulates the overall theme of the chat.
"""},
                  {"role": "user", "content": chat_text}],
        max_tokens=4096,
        temperature=0.1,
        stop=["\n\n"]
    )
    
    return response.choices[0].message.content

In [99]:
def title_for_summary(chat_text, gpt):
    # Using the chat-specific completion API
    response = gpt.chat.completions.create(
        model="gpt-3.5-turbo",  # Use an appropriate model, like 'gpt-3.5-turbo'
        messages=[{"role": "system", "content": """
Create a short one-line title for the summary provided below.
"""},
                  {"role": "user", "content": chat_text}],
        max_tokens=250,
        temperature=0.1,
        stop=["\n\n"]
    )
    
    return response.choices[0].message.content

In [95]:
summarries = []

for transcript in transcripts:
    summarries.append(summarize_chat(transcript, gpt))

In [96]:
print(summarries[0])

The chat discussion revolved around the implications of code immutability, particularly in the context of contracts and libraries in the blockchain. Participants like vb271828 highlighted the value of code immutability for contracts to reference specific addresses on-chain. The conversation delved into the potential risks of accessing mutable libraries, such as in ERC4337, where it could lead to a Denial of Service (DoS) attack vector.


In [100]:
title_for_summary(summarries[0], gpt)

'Implications of Code Immutability in Blockchain Contracts and Libraries.'

In [106]:
def generate_tldr_and_save(engine, start_time: datetime, end_time: datetime, gpt):
    # Reconstruct chat as conversations
    conversations = reconstruct_chat_as_conversations(engine, start_time, end_time, gpt)
    
    # Create transcripts from conversations
    transcripts = create_transcripts_from_conversations(conversations, engine)
    
    # Summarize each transcript
    tldr = {"metadata": {"start_time": start_time, "end_time": end_time}, "data": []}
    for transcript in transcripts:
        summary = summarize_chat(transcript, gpt)
        title = title_for_summary(summary, gpt)
        tldr["data"].append({"title": title, "summary": summary, "transcript": transcript})

    return tldr

In [107]:
# test
tldr = generate_tldr_and_save(engine, start_time, end_time, gpt)

2024-05-13 17:19:30,798 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-13 17:19:30,798 INFO sqlalchemy.engine.Engine SELECT message.id, message.message_id, message.channel_chat_created, message.chat_id, message.date, message.delete_chat_photo, message.from_user_id, message.group_chat_created, message.reply_to_message_id, message.supergroup_chat_created, message.text 
FROM message 
WHERE message.date >= %(date_1)s AND message.date <= %(date_2)s ORDER BY message.date
2024-05-13 17:19:30,799 INFO sqlalchemy.engine.Engine [cached since 1.668e+04s ago] {'date_1': 1715122800.0, 'date_2': 1715554800.0}
2024-05-13 17:19:30,801 INFO sqlalchemy.engine.Engine SELECT "user".id AS user_id_1, "user".user_id AS user_user_id, "user".first_name AS user_first_name, "user".last_name AS user_last_name, "user".is_bot AS user_is_bot, "user".language_code AS user_language_code, "user".username AS user_username 
FROM "user" 
WHERE "user".id = %(pk_1)s
2024-05-13 17:19:30,801 INFO sqlalchemy.engine.Eng

In [108]:
import pprint

pprint.pprint(tldr)

{'data': [{'summary': 'The chat discussion revolved around the concept of code '
                      'immutability in smart contracts, particularly in the '
                      'context of Ethereum Improvement Proposals (EIPs) such '
                      'as EIP-4337 and EIP-7702. Participants like vb271828 '
                      'highlighted the importance of code immutability for '
                      'other contracts to reference specific addresses on the '
                      'chain as libraries. There were concerns raised about '
                      'accessing mutable libraries becoming a potential Denial '
                      'of Service (DoS) attack vector.',
           'title': 'Discussion on Code Immutability in Smart Contracts and '
                    'EIPs: Importance, Concerns, and DoS Attack Vectors.',
           'transcript': 'vb271828: The main value of code immutability is so '
                         'that other contracts can point to specific addresses

In [117]:
print(tldr["data"][0]["title"] + "\n" + tldr["data"][0]["summary"])

Discussion on Code Immutability in Smart Contracts and EIPs: Importance, Concerns, and DoS Attack Vectors.
The chat discussion revolved around the concept of code immutability in smart contracts, particularly in the context of Ethereum Improvement Proposals (EIPs) such as EIP-4337 and EIP-7702. Participants like vb271828 highlighted the importance of code immutability for other contracts to reference specific addresses on the chain as libraries. There were concerns raised about accessing mutable libraries becoming a potential Denial of Service (DoS) attack vector.


In [125]:
def format_tldr(tldr):
    # Format the TLDR start date into a readable string
    formatted_tldr = f"{tldr['metadata']['start_time'].strftime('%B %d, %Y')}\n\n"

    for chat in tldr["data"]:
        formatted_tldr += f"{chat['title']}:\n"
        formatted_tldr += f"{chat['summary']}\n"
        formatted_tldr += f"---\n"

    return formatted_tldr

In [126]:
print(format_tldr(tldr))

May 08, 2024

Discussion on Code Immutability in Smart Contracts and EIPs: Importance, Concerns, and DoS Attack Vectors.:
The chat discussion revolved around the concept of code immutability in smart contracts, particularly in the context of Ethereum Improvement Proposals (EIPs) such as EIP-4337 and EIP-7702. Participants like vb271828 highlighted the importance of code immutability for other contracts to reference specific addresses on the chain as libraries. There were concerns raised about accessing mutable libraries becoming a potential Denial of Service (DoS) attack vector.
---
Comparison of EIP-3074 and EIP-7702 Implications for Wallets and Smart Contracts.:
The chat discussion revolves around the comparison between EIP-3074 and EIP-7702, focusing on the implications for wallets and smart contracts. Thogard expresses a shift in perspective, noting that while the whitelisting issue persists for wallets, smart contracts no longer have on-chain functionality discrepancies due to EIP